# Load environment variables

1, Create a `.env` file from the root directory of the project

2, Add the GEMINI_API_KEY variable

In [1]:
from dotenv import load_dotenv
from typing import List

load_dotenv()

True

# Sample

## Gemini call

Calling the Gemini model to generate an output with a calendar event schema using `call_gemini_calendar_event()`

In [2]:
from agentic_workspace.llm_clients.gemini_calls import call_gemini_calendar_event
from agentic_workspace.models.calendar_event import CalendarEventModel


prompt = "Create a calendar event for the next 3 days, hôm nay là 5/4/2025"

response : List[CalendarEventModel] = call_gemini_calendar_event(prompt)

for event in response:
    print(event.model_dump())


{'text': 'Meeting with team', 'summary': 'Team meeting', 'description': 'Discuss project progress and upcoming tasks.', 'start': {'dateTime': '2025-04-05T10:00:00+07:00', 'timeZone': 'Asia/Ho_Chi_Minh'}, 'end': {'dateTime': '2025-04-05T11:00:00+07:00', 'timeZone': 'Asia/Ho_Chi_Minh'}}
{'text': 'Client presentation', 'summary': 'Presentation to client', 'description': 'Present the final design.', 'start': {'dateTime': '2025-04-06T14:00:00+07:00', 'timeZone': 'Asia/Ho_Chi_Minh'}, 'end': {'dateTime': '2025-04-06T15:00:00+07:00', 'timeZone': 'Asia/Ho_Chi_Minh'}}
{'text': 'Project review', 'summary': 'Review project deliverables', 'description': 'Check and review all deliverables.', 'start': {'dateTime': '2025-04-07T09:00:00+07:00', 'timeZone': 'Asia/Ho_Chi_Minh'}, 'end': {'dateTime': '2025-04-07T10:00:00+07:00', 'timeZone': 'Asia/Ho_Chi_Minh'}}


# Call google calendar API

In [3]:
from googleapiclient.discovery import build
from agentic_workspace.tools.google_calendar.auth import get_calendar_credentials

# Get credentials
creds = get_calendar_credentials()

# Build the Google Calendar API service
service = build('calendar', 'v3', credentials=creds)

# Example: List the next 10 events on the user's calendar
print('Getting the upcoming 10 events')
calendar_id = 'primary'  # Use 'primary' for the user's primary calendar

# Call the Calendar API
events_result = service.events().list(
    calendarId=calendar_id,
    timeMin='2024-01-01T00:00:00Z',  # Start time
    maxResults=10,
    singleEvents=True,
    orderBy='startTime'
).execute()

events = events_result.get('items', [])

if not events:
    print('No upcoming events found.')
else:
    for event in events:
        start = event['start'].get('dateTime', event['start'].get('date'))
        print(f"{start} - {event['summary']}")

Getting the upcoming 10 events
2024-01-01T09:00:00+07:00 - abc
2024-03-24T03:00:00+07:00 - Họp gia đình
2024-03-24T22:00:00+07:00 - Daily
2024-06-11T07:00:00+07:00 - Học tiếng Anh
2024-06-15T01:00:00+07:00 - Trình bày machine learning trên lớp
2024-07-18T11:00:00+07:00 - GenAI Startup event
2024-07-18T13:30:00+07:00 - GenAI Startup event
2024-07-31T15:00:00+07:00 - Thi IELTS
2024-09-16T16:00:23+07:00 - [FPT Telecom] Thư mời phỏng vấn_Data Engineer
2024-09-19T09:00:35+07:00 - [FPT Telecom] Thư mời phỏng vấn vòng 2_Data Engineer


# Insert event

In [3]:
from googleapiclient.discovery import build
from agentic_workspace.tools.google_calendar.auth import get_calendar_credentials

# Get credentials
creds = get_calendar_credentials()

# Build the Google Calendar API service
service = build('calendar', 'v3', credentials=creds)


for event in response:
    try:
        # Insert the event
        event = service.events().insert(
            calendarId='primary',
            body=event.model_dump(),
        ).execute()
        print(f'Event created successfully: {event.get("htmlLink")}')

    except Exception as e:
        print(f'An error occurred: {e}')


Event created successfully: https://www.google.com/calendar/event?eid=MW9obnVqa2tlZDhmbDFranA4dDlvdmlybzAgdG90aGFuaDFmZWIzLnF1aW5uQG0
Event created successfully: https://www.google.com/calendar/event?eid=Zmp1aXNtZDMxcWhibGY0ZGh1cGt1Z2NxcGcgdG90aGFuaDFmZWIzLnF1aW5uQG0
Event created successfully: https://www.google.com/calendar/event?eid=bnMyZTdnaThiZmNucXFqb3VobnRicjg4ZDggdG90aGFuaDFmZWIzLnF1aW5uQG0
